# **AI Workshop: From Agents to Intelligent Search**

## - Led by Ayaan Ahmad

### A hands-on guide on building and programming intelligent agents to solve complex problems.

# AI Workshop - Lab 3: Breadth-First Search (BFS)

Now we'll implement BFS. It explores level-by-level, guaranteeing the shortest path.

We will:
1. Load our standard maze.
2. Run the BFS algorithm.
3. Measure its performance (time, explored cells, path length).
4. Append these metrics to `performance_results.csv`.
5. Visualize the extensive search path versus the optimal final path.

## Install Libraries

In [ ]:
pip install Pillow

### Importing Libraries

In [1]:
from pyamaze import maze, agent, COLOR
import os
import glob
from PIL import Image
from queue import PriorityQueue
import time # To measure execution time
import pandas as pd # To handle CSV data
import matplotlib.pyplot as plt # For plotting
import seaborn as sns # For better visualizations

# Create a list to store performance results
performance_results = []

### 2.2 - Breadth-First Search (BFS)

BFS explores all neighbor nodes at the present depth prior to moving on to the nodes at the next depth level. It uses a FIFO (First-In, First-Out) strategy, like a queue at a grocery store. The first person in line is the first person to be served.

In [2]:
def BFS(m):
    start = (m.rows, m.cols)
    explored = [start]
    frontier = [start]
    bfsPath = {}

    while len(frontier) > 0:
        currCell = frontier.pop(0)
        # Goal check removed to allow for full exploration
        if currCell == (1, 1):
            break
        for d in "ESNW":
            if m.maze_map[currCell][d]:
                if d == 'E': childCell = (currCell[0], currCell[1] + 1)
                elif d == 'W': childCell = (currCell[0], currCell[1] - 1)
                elif d == 'N': childCell = (currCell[0] - 1, currCell[1])
                elif d == 'S': childCell = (currCell[0] + 1, currCell[1])
                if childCell in explored: continue
                explored.append(childCell)
                frontier.append(childCell)
                bfsPath[childCell] = currCell
    
    # After full exploration, reconstruct the forward path
    fwdPath = {}
    cell = (1, 1)
    if cell in bfsPath or cell == start:
        while cell != start:
            fwdPath[bfsPath[cell]] = cell
            cell = bfsPath[cell]
            
    # Return the path and the complete list of explored cells
    return fwdPath, explored

In [3]:
# --- Run BFS ---
m = maze()
m.CreateMaze(loadMaze='workshop_maze.csv')

# --- Measure Execution Time ---
start_time = time.time()
final_path, explored_cells = BFS(m)
end_time = time.time()
bfs_time = end_time - start_time
print(f"BFS completed in {bfs_time:.6f} seconds. Path length: {len(final_path)}. Explored cells: {len(explored_cells)}")

time_taken = end_time - start_time
explored_count = len(explored_cells)
path_length = len(final_path)

# Save Metrics to CSV
results_file = 'performance_results.csv'
new_data = {'Algorithm': 'BFS', 'Time (s)': time_taken, 'Explored Cells': explored_count, 'Path Length': path_length}
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
    df = df[df.Algorithm != 'BFS']
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
else:
    df = pd.DataFrame([new_data])
df.to_csv(results_file, index=False)
print(f"BFS performance data saved to '{results_file}'")

# --- 1. Demo the SEARCH path ---
explored_path = {explored_cells[i]: explored_cells[i+1] for i in range(len(explored_cells)-1)}
search_agent = agent(m, footprints=True, color=COLOR.cyan, filled=True, shape='square')
m.tracePath({search_agent: explored_path})

# --- 2. Demo the FINAL path ---
path_agent = agent(m, footprints=True, color=COLOR.red)
m.tracePath({path_agent: final_path})

BFS completed in 0.012537 seconds. Path length: 72. Explored cells: 554
BFS performance data saved to 'performance_results.csv'


In [ ]:
m.run()

Observation: The BFS path is guaranteed to be the shortest in terms of the number of steps, but it had to explore a large area of the maze to find it.